#Описание гипотезы

Проверим два предположения:

Зависит ли вероятность оплаты от выбранного пользователем уровня сложности бесплатных тренировок?
Существует ли разница во времени между пользователями с разным уровнем сложности и их первой оплатой?
Проверку будем производить на основе данных пользователей, которые зарегистрировались в 2018 году.

Ход проверки

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
events = pd.read_csv('/content/drive/My Drive/Projects/Python (Pandas)/Project_1 (level_selection)/7_4_Events.csv', sep=',')
purchase_df = pd.read_csv('/content/drive/My Drive/Projects/Python (Pandas)/Project_1 (level_selection)/purchase.csv', sep=',')
events.head(5)
purchase_df.head(5)

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100


In [4]:
#Выделяем пользователей, которые зарегестрировались в 2018 г.
mask1 = (events.start_time >= '2018-01-01') & \
        (events.start_time < '2019-01-01') & \
        (events.event_type == 'registration')
registered_2018 = events[mask1]['user_id'].values.tolist()
events_2018 = events[events['user_id'].isin(registered_2018)]


#Переименовываем колонки
purchase_df['event_type'] = 'purchase'
events_2018 = events_2018.rename(columns={"id": "event_id"})
purchase_2018 = purchase_df.rename(columns={"id": "purchase_id"})
purchase_2018['event_datetime'] = pd.to_datetime(purchase_2018['event_datetime'])

#Обьединяем таблицу с событиями и оплатой
total_events_df_2018 = pd.concat([events_2018,purchase_2018],sort=False)

#Выделяем группы пользователей в зависимости от выбранного уровня сложности
mask1 = total_events_df_2018['selected_level'] == 'easy'
selected_level_easy_events_df_2018 = total_events_df_2018[mask1]
users_easy_level = selected_level_easy_events_df_2018["user_id"].unique()
print(f"Количество пользователей, выбравших easy-уровень  равно {len(users_easy_level)}")

mask2 = total_events_df_2018['selected_level'] == 'hard'
selected_level_hard_events_df_2018 = total_events_df_2018[mask2]
users_hard_level = selected_level_hard_events_df_2018["user_id"].unique()
print(f"Количество пользователей, выбравших hard-уровень  равно {len(users_hard_level)}")

mask3 = total_events_df_2018['selected_level'] == 'medium'
selected_level_medium_events_df_2018 = total_events_df_2018[mask3]
users_medium_level = selected_level_medium_events_df_2018["user_id"].unique()
print(f"Количество пользователей, выбравших medium-уровень  равно {len(users_medium_level)}")


#Подсчет процента оплативших пользователей в зависимости от выбранного уровня сложности
purchase_df_easy_level = purchase_df[purchase_df["user_id"].isin(users_easy_level)]
percent_of_purchase_easy_level = round(((purchase_df_easy_level["user_id"].nunique() / len(
    users_easy_level))*100),2)
print(f"Количество пользователей, выбравших easy-уровень и заплативших от общего числа пользователей,"
      f" выбравших уровень  равно {percent_of_purchase_easy_level} %")

purchase_df_hard_level = purchase_df[purchase_df["user_id"].isin(users_hard_level)]
percent_of_purchase_hard_level = round(((purchase_df_hard_level["user_id"].nunique() / len(
    users_hard_level))*100),2)
print(f"Количество пользователей, выбравших hard-уровень и заплативших от общего числа пользователей,"
      f" выбравших уровень  равно {percent_of_purchase_hard_level} %")

purchase_df_medium_level = purchase_df[purchase_df["user_id"].isin(users_medium_level)]
percent_of_purchase_medium_level = round(((purchase_df_medium_level["user_id"].nunique() / len(
    users_medium_level))*100),2)
print(f"Количество пользователей, выбравших medium-уровень и заплативших от общего числа пользователей,"
      f" выбравших уровень  равно {percent_of_purchase_medium_level} %")

#Подсчет среднего времени между событиями оплаты и выбором уровня сложности

level_choice_df = total_events_df_2018[(total_events_df_2018['event_type'] == 'level_choice') & (total_events_df_2018['user_id'].isin(registered_2018))]
if level_choice_df['user_id'].value_counts().mean() == 1:
    selected_level_easy = selected_level_easy_events_df_2018[['user_id', 'start_time']].copy()
    selected_level_easy['start_time'] = pd.to_datetime(selected_level_easy['start_time'])
    selected_and_purchase_level_easy = purchase_df[purchase_df["user_id"].isin(users_easy_level)].copy()
    selected_and_purchase_level_easy['event_datetime'] = pd.to_datetime(selected_and_purchase_level_easy['event_datetime'])
    merged_easy_df = selected_level_easy.merge(selected_and_purchase_level_easy, on="user_id", how="inner")
    merged_easy_df["timedelta"] = merged_easy_df["event_datetime"] - merged_easy_df["start_time"]
    print(f"Среднее время между событиями оплаты и выбором уровня сложности для пользователей, выбравших easy-уровень"
          f"  {merged_easy_df['timedelta'].mean()}")

    selected_level_medium = selected_level_medium_events_df_2018[["user_id", "start_time"]].copy()
    selected_level_medium['start_time'] = pd.to_datetime(selected_level_medium['start_time'])
    selected_and_purchase_level_medium = purchase_df[purchase_df["user_id"].isin(users_medium_level)].copy()
    selected_and_purchase_level_medium['event_datetime'] = pd.to_datetime(selected_and_purchase_level_medium['event_datetime'])
    merged_medium_df = selected_level_medium.merge(selected_and_purchase_level_medium, on="user_id", how="inner")
    merged_medium_df["timedelta"] = merged_medium_df["event_datetime"] - merged_medium_df["start_time"]
    print(f"Среднее время между событиями оплаты и выбором уровня сложности для пользователей, выбравших medium-уровень"
          f"  {merged_medium_df['timedelta'].mean()}")

    selected_level_hard = selected_level_hard_events_df_2018[["user_id", "start_time"]].copy()
    selected_level_hard['start_time'] = pd.to_datetime(selected_level_hard['start_time'])
    selected_and_purchase_level_hard = purchase_df[purchase_df["user_id"].isin(users_hard_level)].copy()
    selected_and_purchase_level_hard['event_datetime'] = pd.to_datetime(selected_and_purchase_level_hard['event_datetime'])
    merged_hard_df = selected_level_hard.merge(selected_and_purchase_level_hard, on="user_id", how="inner")
    merged_hard_df["timedelta"] = merged_hard_df["event_datetime"] - merged_hard_df["start_time"]
    print(f"Среднее время между событиями оплаты и выбором уровня сложности для пользователей, выбравших hard-уровень"
          f"  {merged_hard_df['timedelta'].mean()}")
else:
        print ('Более 1 события выбора уровня сложности')

Количество пользователей, выбравших easy-уровень  равно 2448
Количество пользователей, выбравших hard-уровень  равно 1249
Количество пользователей, выбравших medium-уровень  равно 4645
Количество пользователей, выбравших easy-уровень и заплативших от общего числа пользователей, выбравших уровень  равно 7.72 %
Количество пользователей, выбравших hard-уровень и заплативших от общего числа пользователей, выбравших уровень  равно 35.39 %
Количество пользователей, выбравших medium-уровень и заплативших от общего числа пользователей, выбравших уровень  равно 20.86 %
Среднее время между событиями оплаты и выбором уровня сложности для пользователей, выбравших easy-уровень  3 days 14:58:52.941798941
Среднее время между событиями оплаты и выбором уровня сложности для пользователей, выбравших medium-уровень  3 days 23:14:13.165118679
Среднее время между событиями оплаты и выбором уровня сложности для пользователей, выбравших hard-уровень  3 days 07:20:41.420814479


#Выводы
Процент оплативших пользователей, выбравших уровень сложности easy: 7.72% Процент оплативших пользователей, выбравших уровень сложности medium: 20.86% Процент оплативших пользователей, выбравших уровень сложности hard: 35.39%

Таким образом, существует прямая зависимость между выбранным уровнем сложности и процент пользователей, которые оплатили тренировки. Пользователи, выбирающие более сложные тренировки, чаще покупают платные тренировки.

Наибольшее время между событием выбора уровня сложности и моментом первой оплаты, тратят пользователи, которые выбрали уровень сложности medium. Для них среднее время составляет 3 дня 23 часа, а медианное время 3 дня 19 часов.

Чуть меньше времени тратят пользователи, выбравшие уровень сложности easy. Для них среднее время составляет 3 дня 14 часов, а медианное время 3 дня 6 часов.

Меньше всего времени на принятие решение о покупке тратят пользователя, выбравшие уровень сложности hard. Для них среднее время составляет 3 дня 7 часов, а медианное время 3 дня 13 часов.